# Prepare the train and the test data

This code does the following:
1. adds the nacis groupings 
2. adds default flags (default next year, default in 2 years, defaulted in the past, will ever default in the future)
3. removes observations for which there was a default within the last 3 years
4. removes NaN according to column selection
5. logs and standardizes data

Important parameters in section below:
chosing which columns matter for NaN: "cols_for_NaN_removal"
chosing target column (what will be your y value): "target_col"


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


In [ ]:
# set the path to the file obtained from cleandata.ipynb
compustat_pre_cleaned_path = "filepath_from_cleandata_notebook_output.csv"

compustat_pre_cleaned = pd.read_csv(compustat_pre_cleaned_path)
compustat_pre_cleaned.info()

# Naics groupings

In [ ]:
def add_naics_industry_group(df, naics_col='naicsh'):
    """
    Add industry group classification based on 2-digit NAICS codes.
    
    Parameters:
    -----------
    df : pandas DataFrame
        Must contain the naics_col column
    naics_col : str, default 'naicsh'
        Column name containing NAICS codes
    
    Returns:
    --------
    pandas DataFrame with added columns:
        - naics_2_digit: First 2 digits of NAICS code
        - naics_industry_group: Industry group classification string
    
    Industry Groups:
    ----------------
    - Capital Intensive: 11, 21, 22, 23, 31-33, 48-49
    - Trade: 42, 44-45
    - Financial and Real Estate: 52, 53
    - Administrative: 54, 55, 56, 81, 92
    - Healthcare and Education: 61, 62
    - Food Services and Entertainment: 71, 72
    - Other: All other codes
    """
    
    df = df.copy()
    
    # Extract first 2 digits of NAICS code
    # Handle float (e.g., 523100.0) by converting to int first, then string
    df['naics_2_digit'] = (
        df[naics_col]
        .apply(lambda x: str(int(x))[:2] if pd.notna(x) else np.nan)
    )
    
    # Convert to numeric for easier comparison
    df['_naics_2_numeric'] = pd.to_numeric(df['naics_2_digit'], errors='coerce')
    
    # Define industry group mapping
    def classify_industry(code):
        if pd.isna(code):
            return np.nan
        
        code = int(code)
        
        # Capital Intensive: 11, 21, 22, 23, 31-33, 48-49
        if code in [11, 21, 22, 23] or (31 <= code <= 33) or (48 <= code <= 49):
            return 'Capital Intensive'
        
        # Trade: 42, 44-45
        elif code == 42 or (44 <= code <= 45):
            return 'Trade'
        
        # Financial and Real Estate: 52, 53
        elif code in [52, 53]:
            return 'Financial and Real Estate'
        
        # Administrative: 54, 55, 56, 81, 92
        elif code in [54, 55, 56, 81, 92]:
            return 'Administrative'
        
        # Healthcare and Education: 61, 62
        elif code in [61, 62]:
            return 'Healthcare and Education'
        
        # Food Services and Entertainment: 71, 72
        elif code in [71, 72]:
            return 'Food Services and Entertainment'
        
        else:
            return 'Other'
    
    df['naics_industry_group'] = df['_naics_2_numeric'].apply(classify_industry)
    
    # Drop helper column
    df = df.drop(columns=['_naics_2_numeric'])
    
    # Print summary
    print("NAICS Industry Group Summary:")
    print("-" * 60)
    
    total = len(df)
    non_null = df['naics_industry_group'].notna().sum()
    
    print(f"Total observations: {total:,}")
    print(f"Observations with NAICS code: {non_null:,}")
    print(f"Missing NAICS: {total - non_null:,}")
    print()
    
    group_counts = df['naics_industry_group'].value_counts(dropna=False)
    print("Distribution by industry group:")
    for group, count in group_counts.items():
        pct = count / total * 100
        group_name = group if pd.notna(group) else 'Missing'
        print(f"  {group_name}: {count:,} ({pct:.1f}%)")
    
    return df


In [ ]:
compustat_pre_cleaned = add_naics_industry_group(compustat_pre_cleaned, naics_col='naicsh')

# Default flags

In [ ]:
def add_default_flags(df, default_indicator='TL_flag', default_value=1):
    """
    Add multiple default-related columns to the dataset.

    Parameters
    ----------
    df : pandas DataFrame
        Must contain columns: gvkey, fyear, and the default_indicator column
    default_indicator : str, default 'TL_flag'
        Column name that indicates default status (1 = default, 0 = no default)
    default_value : int or float, default 1
        Value in default_indicator that represents a default event

    Returns
    -------
    pandas DataFrame with added columns:
        - default_next_1y
        - default_next_2y
        - default_past_3y_plus
        - default_ever_future
    """

    df = df.copy()
    df = df.sort_values(['gvkey', 'fyear']).reset_index(drop=True)

    # ============================================================
    # Helper: all default events
    # ============================================================
    defaults = (
        df.loc[df[default_indicator] == default_value, ['gvkey', 'fyear']]
        .rename(columns={'fyear': 'default_year'})
    )

    # ============================================================
    # 1) DEFAULT IN NEXT 1 YEAR (fyear + 1)
    # ============================================================
    next_1y = (
        df[['gvkey', 'fyear', default_indicator]]
        .assign(fyear=lambda x: x['fyear'] - 1)
        .rename(columns={default_indicator: 'default_next_1y'})
    )

    df = df.merge(next_1y, on=['gvkey', 'fyear'], how='left')

    df['default_next_1y'] = np.where(
        df['default_next_1y'].isna(),
        np.nan,
        (df['default_next_1y'] == default_value).astype(float)
    )

    # ============================================================
    # 2) DEFAULT IN NEXT 2 YEARS (fyear + 1 OR fyear + 2)
    # ============================================================
    y1 = (
        df[['gvkey', 'fyear', default_indicator]]
        .assign(fyear=lambda x: x['fyear'] - 1)
        .rename(columns={default_indicator: 'y1'})
    )

    y2 = (
        df[['gvkey', 'fyear', default_indicator]]
        .assign(fyear=lambda x: x['fyear'] - 2)
        .rename(columns={default_indicator: 'y2'})
    )

    df = df.merge(y1, on=['gvkey', 'fyear'], how='left')
    df = df.merge(y2, on=['gvkey', 'fyear'], how='left')

    has_any_year = df[['y1', 'y2']].notna().any(axis=1)
    has_default = (
        (df['y1'] == default_value) |
        (df['y2'] == default_value)
    )

    df['default_next_2y'] = np.where(
        ~has_any_year,
        np.nan,
        has_default.astype(float)
    )

    df = df.drop(columns=['y1', 'y2'])

    # ============================================================
    # 3) DEFAULT MORE THAN 3 YEARS AGO (fyear - 4 or earlier)
    # ============================================================
    past = df.merge(defaults, on='gvkey', how='left')

    past['past_default'] = (past['fyear'] - past['default_year']) > 3

    past_flag = (
        past.groupby(['gvkey', 'fyear'])['past_default']
        .any()
        .astype(float)
        .reset_index(name='default_past_3y_plus')
    )

    df = df.merge(past_flag, on=['gvkey', 'fyear'], how='left')
    df['default_past_3y_plus'] = df['default_past_3y_plus'].fillna(0.0)

    # ============================================================
    # 4) DEFAULT EVER IN FUTURE (any year > fyear)
    # ============================================================
    future = df[['gvkey', 'fyear']].merge(defaults, on='gvkey', how='left')

    future['future_default'] = (future['default_year'] > future['fyear'])

    future_flag = (
        future.groupby(['gvkey', 'fyear'])['future_default']
        .any()
        .reset_index(name='default_ever_future')
    )

    df = df.merge(future_flag, on=['gvkey', 'fyear'], how='left')

    # Identify whether future years exist (not just defaults)
    max_fyear = df.groupby('gvkey')['fyear'].transform('max')

    df['default_ever_future'] = np.where(
        df['fyear'] == max_fyear,
        np.nan,                      # no future years exist
        df['default_ever_future'].astype(float)
    )

    # ============================================================
    # SUMMARY (optional diagnostic output)
    # ============================================================
    print("Default Flags Summary:")
    print("-" * 60)

    for col in [
        'default_next_1y',
        'default_next_2y',
        'default_past_3y_plus',
        'default_ever_future'
    ]:
        non_null = df[col].notna().sum()
        defaults_n = (df[col] == 1).sum()
        rate = defaults_n / non_null * 100 if non_null > 0 else 0

        print(f"{col}:")
        print(f"  Non-null observations: {non_null:,}")
        print(f"  Defaults (=1): {defaults_n:,} ({rate:.2f}%)\n")

    return df


In [ ]:
compustat_pre_cleaned = add_default_flags(compustat_pre_cleaned, default_indicator='TL_flag', default_value=1)

# Removing periods between defaults

In [ ]:
def remove_post_default_observations(df, default_indicator='TL_flag', default_value=1, lookback_years=3):
    """
    Remove observations if the firm had a default in the previous N years.
    
    Parameters:
    -----------
    df : pandas DataFrame
        Must contain columns: gvkey, fyear, and the default_indicator column
    default_indicator : str, default 'TL_flag'
        Column name that indicates default status
    default_value : int/float, default 1
        Value in default_indicator that represents a default event
    lookback_years : int, default 3
        Number of years to look back for prior defaults
    
    Returns:
    --------
    pandas DataFrame with post-default observations removed
    
    Notes:
    ------
    - For each observation, checks if the firm had a default in any of the
      previous `lookback_years` years (fyear-1 through fyear-lookback_years)
    - Removes the observation if any prior default is found
    - The default year itself is NOT removed (only subsequent years)
    - Handles gaps in data correctly (only looks at years that exist)
    
    Example (lookback_years=5):
    --------
    If firm A has:
        fyear=2018, TL_flag=0  -> KEEP (no prior default)
        fyear=2019, TL_flag=1  -> KEEP (this is the default year)
        fyear=2020, TL_flag=0  -> REMOVE (default in 2019, within 5 years)
        fyear=2021, TL_flag=0  -> REMOVE (default in 2019, within 5 years)
        fyear=2022, TL_flag=0  -> REMOVE (default in 2019, within 5 years)
        fyear=2023, TL_flag=0  -> REMOVE (default in 2019, within 5 years)
        fyear=2024, TL_flag=0  -> REMOVE (default in 2019, within 5 years)
        fyear=2025, TL_flag=0  -> KEEP (2019 is now 6 years ago)
    """
    
    df = df.copy()
    
    # Sort by company and fiscal year
    df = df.sort_values(['gvkey', 'fyear']).reset_index(drop=True)
    
    # Get all default events
    defaults = df[df[default_indicator] == default_value][['gvkey', 'fyear']].copy()
    defaults = defaults.rename(columns={'fyear': 'default_year'})
    
    # For each observation, check if there's a default in the lookback window
    # Merge all defaults for each company
    df_with_defaults = df.merge(defaults, on='gvkey', how='left')
    
    # Calculate years since default
    df_with_defaults['years_since_default'] = df_with_defaults['fyear'] - df_with_defaults['default_year']
    
    # Flag observations where a default occurred in the lookback window
    # (years_since_default between 1 and lookback_years, inclusive)
    df_with_defaults['has_recent_default'] = (
        (df_with_defaults['years_since_default'] >= 1) & 
        (df_with_defaults['years_since_default'] <= lookback_years)
    )
    
    # Group by original observation and check if ANY prior default exists
    recent_default_flag = df_with_defaults.groupby(['gvkey', 'fyear'])['has_recent_default'].any().reset_index()
    recent_default_flag = recent_default_flag.rename(columns={'has_recent_default': 'remove_flag'})
    
    # Merge back to original dataframe
    df = df.merge(recent_default_flag, on=['gvkey', 'fyear'], how='left')
    
    # Fill NaN (firms with no defaults at all) with False
    df['remove_flag'] = df['remove_flag'].fillna(False)
    
    # Print summary before removal
    total_obs = len(df)
    obs_to_remove = df['remove_flag'].sum()
    firms_affected = df[df['remove_flag']]['gvkey'].nunique()
    
    print("Post-Default Observation Removal Summary:")
    print("-" * 50)
    print(f"Lookback window: {lookback_years} years")
    print(f"Total observations before: {total_obs:,}")
    print(f"Observations to remove: {obs_to_remove:,}")
    print(f"Firms affected: {firms_affected:,}")
    print(f"Observations remaining: {total_obs - obs_to_remove:,}")
    print(f"Percentage removed: {obs_to_remove / total_obs * 100:.2f}%")
    
    # Remove flagged observations
    df_clean = df[~df['remove_flag']].drop(columns=['remove_flag']).reset_index(drop=True)
    
    return df_clean

In [ ]:
compustat_pre_cleaned = remove_post_default_observations(compustat_pre_cleaned, default_indicator='TL_flag', default_value=1, lookback_years=3)

# Remove NaN

Important: in cell 17, define which columns matter for you when removing NaN values. For example, if you do not care about missing values in columns like "revt_5_year_pct_change", do not include it in the list. 

The function remove_nan_rows takes in a list of columns as an argument, and removes all rows, which have a NaN entry in at least one of the specified columns.

In [ ]:
def remove_nan_rows(df, cols, how='any'):
    """
    Remove rows containing NaN in specified columns.
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataset
    cols : list
        List of column names to check for NaN
    how : str, default 'any'
        - 'any': Remove row if ANY of the specified columns has NaN
        - 'all': Remove row only if ALL of the specified columns have NaN
    
    Returns:
    --------
    pandas DataFrame with rows removed
    
    Example:
    --------
    # Remove rows where any of these columns is NaN
    df_clean = remove_nan_rows(df, cols=['revt', 'at', 'ni'], how='any')
    
    # Remove rows only where ALL of these columns are NaN
    df_clean = remove_nan_rows(df, cols=['revt', 'at', 'ni'], how='all')
    """
    
    # Filter to columns that exist
    cols_exist = [c for c in cols if c in df.columns]
    cols_missing = [c for c in cols if c not in df.columns]
    
    if cols_missing:
        print(f"Warning: Columns not found (ignored): {cols_missing}")
    
    if not cols_exist:
        print("No valid columns specified. Returning original DataFrame.")
        return df
    
    # Count NaN before removal
    rows_before = len(df)
    
    # Per-column NaN counts
    print("NaN counts per column before removal:")
    print("-" * 40)
    for col in cols_exist:
        nan_count = df[col].isna().sum()
        pct = nan_count / rows_before * 100
        print(f"  {col}: {nan_count:,} ({pct:.1f}%)")
    
    # Remove rows
    df_clean = df.dropna(subset=cols_exist, how=how).reset_index(drop=True)
    
    rows_after = len(df_clean)
    rows_removed = rows_before - rows_after
    
    # Summary
    print()
    print(f"Removal method: '{how}'")
    print("-" * 40)
    print(f"Rows before: {rows_before:,}")
    print(f"Rows removed: {rows_removed:,} ({rows_removed / rows_before * 100:.1f}%)")
    print(f"Rows after: {rows_after:,}")
    
    return df_clean

In [ ]:
cols_for_NaN_removal = [
    'revt',    # Total Revenue
    'at',      # Total Assets
    'ni',      # Net Income
    'lt',      # Total Liabilities
    'ceq',     # Common Equity
    'oancf'    # Operating Activities Cash Flow
]

compustat_pre_cleaned = remove_nan_rows(
    compustat_pre_cleaned,
    cols_for_NaN_removal
)

# Logs, standardize, and train/test split

important: check which variables are standardized/logged in cell 21, and let me know if you think something should be changed.

In [ ]:
def signed_log_transform(df, cols):
    """Apply signed log transformation: sign(x) * log(1 + |x|)"""
    df = df.copy()
    cols_exist = [c for c in cols if c in df.columns]
    for col in cols_exist:
        df[col] = np.sign(df[col]) * np.log1p(np.abs(df[col]))
    return df


def standardize(df, cols, scaling_params=None):
    """
    Standardize columns. If scaling_params provided, use those (for test data).
    Otherwise compute from data (for training data) and return params.
    """
    df = df.copy()
    cols_exist = [c for c in cols if c in df.columns]
    
    if scaling_params is None:
        # Training mode: compute parameters
        scaling_params = {}
        for col in cols_exist:
            mean, std = df[col].mean(), df[col].std()
            std = std if std != 0 else 1
            df[col] = (df[col] - mean) / std
            scaling_params[col] = {'mean': mean, 'std': std}
        return df, scaling_params
    else:
        # Test mode: use provided parameters
        for col in cols_exist:
            if col in scaling_params:
                mean = scaling_params[col]['mean']
                std = scaling_params[col]['std']
                df[col] = (df[col] - mean) / std
        return df


def prepare_data(df, feature_cols, target_col, log_cols, standardize_cols,
                 test_size=0.2, random_state=12):
    """
    Prepare data for SVM training.
    
    Parameters:
    -----------
    df : pandas DataFrame
    feature_cols : list
        All feature column names
    target_col : str
        Target variable name (e.g., 'default_next_period')
    log_cols : list
        Columns to apply signed log transform
    standardize_cols : list
        Columns to standardize
    test_size : float
        Proportion of data for test set
    random_state : int
        Random seed for reproducibility
    
    Returns:
    --------
    tuple: (train_df, test_df, scaling_params)
    """
    
    df = df.copy()
    
    # Split into train/test
    train_df, test_df = train_test_split(
        df, test_size=test_size, random_state=random_state,
        stratify=df[target_col]
    )
    
    print(f"Training set: {len(train_df):,}")
    print(f"Test set: {len(test_df):,}")
    print(f"Default rate (train): {train_df[target_col].mean():.2%}")
    print(f"Default rate (test): {test_df[target_col].mean():.2%}")
    
    # Apply log transform
    train_df = signed_log_transform(train_df, log_cols)
    test_df = signed_log_transform(test_df, log_cols)
    
    # Standardize (fit on train, apply to test)
    train_df, scaling_params = standardize(train_df, standardize_cols)
    test_df = standardize(test_df, standardize_cols, scaling_params)
    
    return train_df, test_df, scaling_params

In [ ]:
raw_level_cols = ['revt', 'oibdp', 'ni', 'xint', 'dp', 'at', 'act', 'che',
                  'lt', 'lct', 'dlc', 'dltt', 'ceq', 'csho', 'dv', 'total_debt',
                  'cogs','ebit','re','rect','invt','oancf', 'capx', 'fincf', 'ivncf','prcc_f', 'market_capitalization']

ratio_cols = ['gross_margin', 'net_profit_margin', 'roa', 'roe', 'asset_turnover',
              'cash_to_assets', 'fixed_asset_intensity', 'current_ratio', 'quick_ratio',
              'debt_to_assets', 'debt_to_equity', 'liabilities_to_assets',
              'interest_coverage', 'long_term_debt_ratio', 'book_value_per_share',
              'earnings_per_share', 'dividend_payout_ratio', 'dividend_yield', 'retention_ratio',
              'capex_to_assets','investing_cash_flow_to_assets','days_sales_outstanding','days_inventory_outstanding',
              'net_working_capital_to_assets','free_cash_flow_to_sales','financing_cash_flow_to_assets']

growth_cols = ['revt_1_year_pct_change', 'at_1_year_pct_change', 'ni_1_year_pct_change',
               'oibdp_1_year_pct_change', 'ceq_1_year_pct_change', 'total_debt_1_year_pct_change',
               'revt_2_year_pct_change', 'at_2_year_pct_change', 'ni_2_year_pct_change',
               'oibdp_2_year_pct_change', 'ceq_2_year_pct_change', 'total_debt_2_year_pct_change',
               'revt_5_year_pct_change', 'at_5_year_pct_change', 'ni_5_year_pct_change',
               'oibdp_5_year_pct_change', 'ceq_5_year_pct_change', 'total_debt_5_year_pct_change',
               'capx_1_year_pct_change','market_capitalization_1_year_pct_change','free_cash_flow_to_sales_1_year_pct_change','financing_cash_flow_to_assets_1_year_pct_change',
               'capx_2_year_pct_change','market_capitalization_2_year_pct_change','free_cash_flow_to_sales_2_year_pct_change','financing_cash_flow_to_assets_2_year_pct_change',
               'capx_5_year_pct_change','market_capitalization_5_year_pct_change','free_cash_flow_to_sales_5_year_pct_change','financing_cash_flow_to_assets_5_year_pct_change'
                ]

feature_cols = raw_level_cols + ratio_cols + growth_cols
log_cols = raw_level_cols  # Only log transform raw levels
standardize_cols = feature_cols  # Standardize all features

chose what is your target columns (is it default in current period, in the t+1, t+2, etc.):

In [ ]:
target_col = 'default_ever_future'

train_set, test_set, scaling_params = prepare_data(compustat_pre_cleaned, feature_cols, target_col, log_cols, standardize_cols)

# Saving test and train sets

In [ ]:
train_set.to_csv(
    "train_set.csv",
    index=False
)

test_set.to_csv(
    "test_set.csv",
    index=False
)